<a href="https://colab.research.google.com/github/tektungg/Machine-Learning/blob/main/Pertemuan%204/Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
# Image directories
train_dir = "/content/drive/MyDrive/Kuliah/Machine Learning/Pertemuan 4/assets/img/training"
test_dir = "/content/drive/MyDrive/Kuliah/Machine Learning/Pertemuan 4/assets/img/test"

In [3]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))

    return img_list

In [5]:
train_img = load_dataset(train_dir)
test_img = load_dataset(test_dir)

In [6]:
pick_random = np.random.randint(0, len(train_img))

print(f'Image {pick_random}')
print(train_img[pick_random][0].shape)

Image 158
(471, 640, 3)


In [7]:
def random_img_viz(img_list):
    rand_num = np.random.randint(0, len(img_list))

    img = img_list[rand_num][0]
    label = img_list[rand_num][1]
    label_str = 'day' if label == 1 else 'night'

    plt.imshow(img)
    print(f'Shape\t: {img.shape}')
    print(f'Label\t: {label}')

In [8]:
def standarized_input(image):
    std_img = cv2.resize(image, (1100,600))

    return std_img

In [9]:
def label_encoder(label):

    num_val = 0

    if(label == 'day'):
        num_val = 1

    return num_val

In [10]:
def preprocess(img_list):
    std_img_list = []

    for item in img_list:
        image = item[0]
        label = item[1]

        std_img = standarized_input(image)

        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

In [11]:
train_std_img_list = preprocess(train_img)
test_std_img_list = preprocess(test_img)

In [15]:
pick_random = np.random.randint(0, len(train_std_img_list))

print(f'Image {pick_random}')
print(train_std_img_list[pick_random][0].shape)

Image 187
(600, 1100, 3)


## Histogram

In [16]:
def extract_color_histogram(image, bins=(8, 8, 8)):
  hsv_img = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  hist = cv2.calcHist([hsv_img], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
  cv2.normalize(hist, hist)

  return hist.flatten()

In [17]:
def extract_histogram_feature(img_list):
    hist_features = []
    labels = []

    for img in img_list:
        hist = extract_color_histogram(img[0])
        img_label = img[1]

        hist_features.append(hist)
        labels.append(img_label)

    df = pd.DataFrame(hist_features)
    df['LABELS'] = labels

    return df

In [18]:
train_hist_img = extract_histogram_feature(train_std_img_list)
print(f'Shape: {train_hist_img.shape}')
train_hist_img.head()

Shape: (240, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,LABELS
0,0.924447,0.154662,0.028831,0.010380,0.007204,0.004479,0.000561,0.002750,0.005676,0.082035,...,0.0,0.003362,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0
1,0.036124,0.000834,0.000285,0.000093,0.000064,0.000036,0.000043,0.000983,0.049235,0.005892,...,0.0,0.039074,0.000770,0.000143,0.000007,0.000000,0.0,0.0,0.0,0
2,0.036840,0.000905,0.000232,0.000072,0.000036,0.000029,0.000022,0.000731,0.052926,0.006023,...,0.0,0.038657,0.000782,0.000080,0.000022,0.000014,0.0,0.0,0.0,0
3,0.996898,0.005227,0.005873,0.002849,0.000591,0.000185,0.000090,0.000383,0.000717,0.036760,...,0.0,0.000047,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0
4,0.888382,0.000239,0.000020,0.000000,0.000000,0.000005,0.000000,0.001180,0.003496,0.004184,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0


In [19]:
test_hist_img = extract_histogram_feature(test_std_img_list)
print(f'Shape: {test_hist_img.shape}')
test_hist_img.head()

Shape: (160, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,LABELS
0,0.794804,0.008155,0.001491,0.002547,0.005874,0.005688,0.005883,0.041077,0.170225,0.032576,...,0.0,0.000302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.099042,0.011718,0.003119,0.001625,0.001164,0.001288,0.001439,0.008193,0.012359,0.017371,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.000153,0.008081,0.200989,0.021796,0.002996,0.001448,0.001995,0.008906,0.000789,0.118361,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.879989,0.000101,0.000010,0.000010,0.000005,0.000000,0.000000,0.001085,0.003028,0.003322,...,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.874357,0.000146,0.000090,0.000084,0.000303,0.000375,0.000140,0.001479,0.004465,0.003194,...,0.0,0.000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## SVM

In [21]:
x_train = train_hist_img.iloc[:, :-1].values
y_train = train_hist_img.iloc[:, -1].values
x_test = test_hist_img.iloc[:, :-1].values
y_test = test_hist_img.iloc[:, -1].values

In [22]:
model_hist = SVC(kernel='linear')
model_hist.fit(x_train, y_train)

SVC(kernel='linear')

In [23]:
y_train_pred_hist = model_hist.predict(x_train)
y_test_pred_hist = model_hist.predict(x_test)

In [24]:
acc_train_hist = accuracy_score(y_train, y_train_pred_hist)
acc_test_hist = accuracy_score(y_test, y_test_pred_hist)


print(f'Accuracy on train Histogram: {acc_train_hist}')
print(f'Accuracy on test Histogram: {acc_test_hist}')

Accuracy on train Histogram: 0.9958333333333333
Accuracy on test Histogram: 0.9625
